In [1]:
import math, json, os, sys
from classification_models import ResNet18
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.applications import MobileNetV2
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


# DATA_DIR = 'data'
TRAIN_DIR = "/home/jeniffer/abs/blurDetection/train"
VALID_DIR = "/home/jeniffer/abs/blurDetection/test"
SIZE = (224, 224)
BATCH_SIZE = 64

# Spatial Rich Models kernel initializer function
def srm(shape, dtype=None):
    data = np.load('/home/archimedes/abs/saravanan-rnd/forgery/custom/SRM_Kernels.npy')
    data = np.concatenate((data,data,data), axis = 2)
    return data

class CustomInitializer:
    def __call__(self, shape, dtype=None):
        return srm(shape, dtype=dtype)
    

            
            
if __name__ == "__main__":
    num_train_samples = sum([len(files) for r, d, files in os.walk(TRAIN_DIR)])
    num_valid_samples = sum([len(files) for r, d, files in os.walk(VALID_DIR)])

    num_train_steps = math.floor(num_train_samples/BATCH_SIZE)
    num_valid_steps = math.floor(num_valid_samples/BATCH_SIZE)

    gen = keras.preprocessing.image.ImageDataGenerator()
    val_gen = keras.preprocessing.image.ImageDataGenerator()

    batches = gen.flow_from_directory(TRAIN_DIR, target_size=SIZE, class_mode='categorical', shuffle=True, batch_size=BATCH_SIZE)
    val_batches = val_gen.flow_from_directory(VALID_DIR, target_size=SIZE, class_mode='categorical', shuffle=True, batch_size=BATCH_SIZE)
    classes = list(iter(batches.class_indices))
    model= MobileNetV2(include_top=True, weights='imagenet', input_shape=(224,224,3))
#     print(base_model.summary())
    model.layers.pop()
#     model.layers.pop()
    print(model.summary())
    for layer in model.layers:
        layer.trainable=True
    last = model.layers[-1].output
    x = Dense(len(classes), activation="softmax")(last)
    finetuned_model = Model(model.input, x)
    finetuned_model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
#     x = keras.layers.GlobalAveragePooling2DgePooling2D((7,7))(base_model.output)
#     x = keras.layers.Dropout(0.3)(x)
#     x = Dense(len(classes), activation="softmax")(x)
#     finetuned_model = Model(base_model.input, x)
#     print(finetuned_model.summary())
# #     model.layers.pop()
#     for layer in finetuned_model.layers:
#         layer.trainable=True
#     last = model.layers[-1].output
#     x = Dense(len(classes), activation="softmax")(last)
    
    finetuned_model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    for c in batches.class_indices:
        classes[batches.class_indices[c]] = c
    finetuned_model.classes = classes

    early_stopping = EarlyStopping(patience=10)
    checkpointer = ModelCheckpoint('models/mobilenet_blur_weights_{epoch:02d}.h5', verbose=1, save_best_only=True)

    finetuned_model.fit_generator(batches, steps_per_epoch=num_train_steps, epochs=20, callbacks=[early_stopping, checkpointer], validation_data=val_batches, validation_steps=num_valid_steps)
    #finetuned_model.save('mobilenet_blur.h5')

Using TensorFlow backend.


Found 48000 images belonging to 2 classes.
Found 12000 images belonging to 2 classes.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________

Epoch 1/20
750/750 [==============================] - 575s 767ms/step - loss: 0.0210 - acc: 0.9919 - val_loss: 0.0025 - val_acc: 0.9994

Epoch 00001: val_loss improved from inf to 0.00254, saving model to mobilenet_blur.h5
Epoch 2/20
750/750 [==============================] - 566s 754ms/step - loss: 0.0038 - acc: 0.9989 - val_loss: 0.0032 - val_acc: 0.9991

Epoch 00002: val_loss did not improve from 0.00254
Epoch 3/20
750/750 [==============================] - 564s 752ms/step - loss: 0.0012 - acc: 0.9997 - val_loss: 2.6427e-04 - val_acc: 0.9999

Epoch 00003: val_loss improved from 0.00254 to 0.00026, saving model to mobilenet_blur.h5
Epoch 4/20
750/750 [==============================] - 563s 751ms/step - loss: 0.0030 - acc: 0.9992 - val_loss: 0.0016 - val_acc: 0.9994

Epoch 00004: val_loss did not improve from 0.00026
Epoch 5/20
750/750 [==============================] - 563s 751ms/step - loss: 0.0012 - acc: 0.9997 - val_loss: 0.0012 - val_acc: 0.9999

Epoch 00005: val_loss did not imp